# Movie Recommendation System using SVD
In this notebook, we'll build a simple movie recommendation system using **Singular Value Decomposition (SVD)**. We'll use the MovieLens dataset to find movies similar to a randomly selected movie based on user ratings.

## Import Libraries

In [29]:
import pandas as pd
import numpy as np
import random
from sklearn.metrics.pairwise import cosine_similarity
from sklearn.decomposition import TruncatedSVD

## Load the Data
First, we'll load the user ratings and movie titles, and then merge them into a single dataframe.

In [30]:
# Load ratings data
column_names = ['user_id', 'item_id', 'rating', 'timestamp']
ratings = pd.read_csv('../data/movielens/u.data', sep='\t', names=column_names)

# Load movie titles
movie_titles = pd.read_csv('../data/movielens/u.item', sep='|', encoding='latin-1',
                           usecols=[0, 1], names=['item_id', 'title'])

# Merge the datasets
data = pd.merge(ratings, movie_titles, on='item_id')

## Explore the Data
Let's take a quick look at the data to understand its structure.

In [5]:
data.head()

,user_id,item_id,rating,timestamp,title
0,196,242,3,881250949,Kolya (1996)
1,63,242,3,875747190,Kolya (1996)
2,226,242,5,883888671,Kolya (1996)
3,154,242,3,879138235,Kolya (1996)
4,306,242,5,876503793,Kolya (1996)


## Create the User-Item Ratings Matrix
We pivot the data to create a matrix where each row represents a user and each column represents a movie. The values are the ratings given by users to movies.

In [10]:
ratings_matrix = data.pivot_table(index='user_id', columns='title', values='rating')

## Fill Missing Values
Since not all users have rated all movies, there will be missing values in the matrix. We'll replace these missing values with zeros to prepare the data for SVD.

In [31]:
ratings_matrix_filled = ratings_matrix.fillna(0)

## Perform Singular Value Decomposition (SVD)
We use SVD to reduce the dimensionality of the ratings matrix and uncover latent features that capture the underlying structure in the data.

In [32]:
# Perform SVD with reduced components
svd = TruncatedSVD(n_components=20)
svd_matrix = svd.fit_transform(ratings_matrix_filled.T)

### Understanding SVD

SVD factorizes the matrix $ A $ into three matrices:

$$
A \approx U \Sigma V^T
$$

- $ U $ : Left singular vectors (movies in latent feature space)
- $ \Sigma $ : Singular values (importance of each latent feature)
- $ V^T $ : Right singular vectors (users in latent feature space)

By reducing the number of components, we capture the most significant patterns in user ratings.

### Build the Recommendation Function

We define a function to find movies similar to a randomly selected movie based on cosine similarity in the reduced feature space.


In [35]:
def get_similar_movies_svd(num_movies=5):
    # Pick a random movie from the list of available titles
    # random_movie = random.choice(ratings_matrix.columns)
    random_movie = 'Indiana Jones and the Last Crusade (1989)'
    print(f"Randomly selected movie: {random_movie}")
    
    # Find the index of the movie in the original ratings matrix
    movie_idx = ratings_matrix.columns.get_loc(random_movie)
    
    # Get the vector for the specified movie in the reduced feature space
    movie_vec = svd_matrix[movie_idx].reshape(1, -1)
    
    # Compute cosine similarity with other movies in the reduced space
    cosine_sim = cosine_similarity(movie_vec, svd_matrix)[0]
    
    # Get indices of the top similar movies
    similar_idx = np.argsort(cosine_sim)[-num_movies-1:-1][::-1]
    
    # Return the titles of the most similar movies
    return ratings_matrix.columns[similar_idx].tolist()


## Get Movie Recommendations
Now, let's get recommendations based on a randomly selected movie.

In [37]:
# Run the function to see recommendations for a random movie
recommended_movies = get_similar_movies_svd(100)
print("Recommended movies:", recommended_movies)

Randomly selected movie: Indiana Jones and the Last Crusade (1989)
Recommended movies: ['Empire Strikes Back, The (1980)', 'Back to the Future (1985)', 'Raiders of the Lost Ark (1981)', 'Groundhog Day (1993)', 'Jurassic Park (1993)', 'Speed (1994)', 'Hunt for Red October, The (1990)', 'Terminator, The (1984)', 'Fugitive, The (1993)', 'Princess Bride, The (1987)', 'Terminator 2: Judgment Day (1991)', 'Blues Brothers, The (1980)', 'Top Gun (1986)', 'Braveheart (1995)', 'Batman (1989)', 'Monty Python and the Holy Grail (1974)', 'E.T. the Extra-Terrestrial (1982)', 'True Lies (1994)', 'Apollo 13 (1995)', 'Dances with Wolves (1990)', 'Forrest Gump (1994)', 'When Harry Met Sally... (1989)', 'Sneakers (1992)', 'Dead Poets Society (1989)', 'Dave (1993)', 'Star Trek: The Wrath of Khan (1982)', 'Aliens (1986)', 'Die Hard (1988)', 'Alien (1979)', 'Maverick (1994)', 'Blade Runner (1982)', 'Get Shorty (1995)', 'Sleepless in Seattle (1993)', 'Return of the Jedi (1983)', 'Pulp Fiction (1994)', 'In th

In [38]:
import pickle

# Save the SVD matrix
with open('../app/models/svd_matrix.pkl', 'wb') as f:
    pickle.dump(svd_matrix, f)

# Save the SVD model (optional, if you need to retrain or use components)
with open('../app/models/svd_model.pkl', 'wb') as f:
    pickle.dump(svd, f)

In [39]:
# Save movie titles
with open('../app/models/movie_titles.pkl', 'wb') as f:
    pickle.dump(ratings_matrix.columns.tolist(), f)

# Save ratings matrix columns (if needed)
with open('../app/models/ratings_matrix_columns.pkl', 'wb') as f:
    pickle.dump(ratings_matrix.columns, f)
